<a href="https://colab.research.google.com/github/1900690/yolo-bbox-area-caliculate/blob/main/area_caliculate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install ultralytics  # install
from ultralytics import YOLO, checks, hub

checks()  # checks

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11n.pt")

model.predict("bus.jpg", save=True, imgsz=320, conf=0.5,save_txt=True)

In [ ]:
#@title ダイレクトカラー画像から害虫の位置をエクセルにまとめる
#ライブラリをインポート
import cv2
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
from skimage import measure
import csv
import glob
import os

#画像の名前を読み込み
read_files_name='/content/instance_segmentations/*'
read_files = glob.glob(read_files_name)

#インデックスカラー化した画像を入れるフォルダを作成
if not os.path.exists('/content/index_coler/'):
  os.mkdir('/content/index_coler/')

#作るエクセルファイルの見出し行を作成
data = [["Name","Type","Value","x1","y1","x2","y2","Attributes","color_number","color_area","center_name"],]
with open("mask_bbox.csv", "w") as file:
        writer = csv.writer(file)
        writer.writerows(data)

#処理を画像の枚数分繰り返す
for file_name in read_files:
  fname = file_name #画像ファイル名
  img = np.array(Image.open(fname))
  NAME=fname[32:-4]

  #numpy形式からpil形式へ変換
  im = Image.fromarray(img)
  #ダイレクトカラーからインデックスカラーへ変換
  c = im.getcolors(im.width * im.height)#この関数はpil形式でしか使えない
  h,w=np.array(im).shape[:2]
  indexcoler=np.zeros((h,w))
  for coler in range(len(c)):
     split=np.zeros((h,w,3))
     split[:,:,0]=np.full((h, w), c[coler][1][0])
     split[:,:,1]=np.full((h, w), c[coler][1][1])
     split[:,:,2]=np.full((h, w), c[coler][1][2])
     truefalse=np.equal(split,im)
     zeroone = np.where(truefalse == True, 1, 0)
     after=zeroone[:,:,0]+zeroone[:,:,1]+zeroone[:,:,2]
     split_after = np.where(after == 3, coler, 0)
     indexcoler=indexcoler+split_after
  #pil形式からnumpy形式へ変換
  img = np.array(indexcoler)

  #色を二値化（白黒の二色にする）
  c = Image.fromarray(img).getcolors(Image.fromarray(img).width * Image.fromarray(img).height)
  img_binary = np.where(img == max(c)[1], 0, 255)

  #color_numberの順番を面積が大きいほうから０を割り振る（インデックス０番は必ず背景色の黒になるようにする）
  img2=np.zeros_like((img))
  for i in range(int(sorted(c,reverse=True)[0][1])+1):
    img2= np.where(img ==int(sorted(c,reverse=True)[i][1]),int((sorted(c,reverse=True)[0][1])-(sorted(c,reverse=True)[i][1])),img2)
  c2 = Image.fromarray(img2).getcolors(Image.fromarray(img2).width * Image.fromarray(img2).height)

  #次のセルで使うためにインデックスカラー画像を保存
  cv2.imwrite("/content/index_coler/"+file_name[32:],img2)

  #各害虫の位置の座標を求める
  #contours = measure.find_contours(img, 0.0)
  contours_binary = measure.find_contours(img_binary, 0.0)

  #座標の情報をエクセルファイルに書き込んでいく
  for number in list(range(len(contours_binary))):

    y_min = int(np.amin(contours_binary[number],axis=0)[0])
    y_max = int(np.amax(contours_binary[number],axis=0)[0])
    x_min = int(np.amin(contours_binary[number],axis=0)[1])
    x_max = int(np.amax(contours_binary[number],axis=0)[1])
    #color_number = img2[int((y_min+y_max)/2),int((x_min+x_max)/2)]
    color_number = np.amax(img2[y_min:y_max+1,x_min:x_max+1])
    coler=str(int(color_number))
    area  = (y_max-y_min)*(x_max-x_min)
    centername=NAME+"_"+str('{0:04d}'.format(int((x_max+x_min)/2)))+"_"+str('{0:04d}'.format(int((y_max+y_min)/2)))
    data=[[NAME,"bbox","mite",x_min,y_min,x_max,y_max,"",coler,area,centername]]

    with open("mask_bbox.csv",mode="a", newline="") as file:
          writer = csv.writer(file)
          writer.writerows(data)